# TFM

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

## Lectura de fichero

In [2]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_dataframe = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_dataframe}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [3]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20247,25",-5580,"6051,35","20718,6","20679,11","-39,49","-2,21",0
1,2,"20679,11",-6195,"7936,54","22420,65","22497,6","76,95","164,76",0
2,3,"22497,6",-6165,"5995,68","22328,28","22284,53","-43,75","73,79",0
3,4,"22284,53",-5720,"5031,76","21596,29","21654,39","58,1","181,92",0
4,5,"21654,39",-6075,"9124,56","24703,95","24903,47","199,52","323,61",0
...,...,...,...,...,...,...,...,...,...
995,996,"14567,32",-5990,"6995,54","15572,86","15611,86","39,0","-8,11",1
996,997,"15611,86",-6405,"6226,8","15433,66","15483,99","50,33","14,33",1
997,998,"15483,99",-5670,"5799,07","15613,06","15537,81","-75,25","6,19",1
998,999,"15537,81",-6130,"5099,01","14506,82","14515,34","8,52","84,27",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17135,63","-5975,0","7990,02","19150,65","19183,49","32,84","106,49",0
1,2,"19183,49","-6310,0","7021,06","19894,55","19852,62","-41,93","88,07",0
2,3,"19852,62","-5855,0","7068,29","21065,91","21015,88","-50,03","36,94",0
3,4,"21015,88","-5995,0","5991,07","21011,95","21025,1","13,15","-10,55",0
4,5,"21025,1","-6045,0","6914,83","21894,93","21927,46","32,53","-67,99",0
...,...,...,...,...,...,...,...,...,...
995,996,"2535,76","-6170,0","6732,66","3098,42","3084,18","-14,24","-244,63",1
996,997,"3084,18","-6050,0","5225,99","2260,17","2291,41","31,24","-67,43",1
997,998,"2291,41","-5735,0","7144,7","3701,11","3650,86","-50,25","-129,83",1
998,999,"3650,86","-5915,0","3212,68","948,54","914,6","-33,94","-248,7",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,22","-5810,0","7001,2","23334,42","23347,02","12,6","-90,27",0
1,2,"23347,02","-6240,0","7270,93","24377,95","24283,54","-94,41","-105,27",0
2,3,"24283,54","-6430,0","3650,03","21503,57","21510,13","6,56","-17,7",0
3,4,"21510,13","-5850,0","6940,97","22601,1","22644,04","42,94","27,91",0
4,5,"22644,04","-6025,0","5910,84","22529,88","22565,92","36,04","80,35",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,99","-0,0","0,0","20,99","7,38","-13,61","5,7",1
996,997,"7,38","-0,0","0,0","7,38","2,99","-4,39","-5,43",1
997,998,"2,99","-0,0","0,0","2,99","14,37","11,38","9,91",1
998,999,"14,37","-0,0","0,0","14,37","23,46","9,09","10,89",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"13689,29","-6200,0","9859,61","17348,9","17315,01","-33,89","93,19",0
1,2,"17315,01","-5965,0","6578,38","17928,39","18121,96","193,57","276,59",0
2,3,"18121,96","-6170,0","5339,44","17291,4","17357,0","65,6","264,32",0
3,4,"17357,0","-5975,0","6966,25","18348,25","18365,14","16,89","200,58",0
4,5,"18365,14","-6135,0","7964,66","20194,8","20222,38","27,58","192,82",0
...,...,...,...,...,...,...,...,...,...
995,996,"14,6","-0,0","0,0","14,6","3,95","-10,65","-1,28",1
996,997,"3,95","-0,0","0,0","3,95","8,56","4,61","-6,84",1
997,998,"8,56","-0,0","0,0","8,56","1,51","-7,05","-2,43",1
998,999,"1,51","-0,0","0,0","1,51","15,58","14,07","12,31",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21372,17","-5985,0","7788,9","23176,07","23214,64","38,57","19,95",0
1,2,"23214,64","-5765,0","2969,27","20418,91","20415,11","-3,8","61,14",0
2,3,"20415,11","-6325,0","7390,24","21480,35","21602,29","121,94","144,09",0
3,4,"21602,29","-6190,0","7315,88","22728,17","22691,65","-36,52","217,46",0
4,5,"22691,65","-5690,0","6031,24","23032,89","22983,03","-49,86","170,15",0
...,...,...,...,...,...,...,...,...,...
995,996,"17,19","-0,0","0,0","17,19","26,77","9,58","23,15",1
996,997,"26,77","-0,0","0,0","26,77","11,55","-15,22","10,12",1
997,998,"11,55","-0,0","0,0","11,55","1,45","-10,1","-1,74",1
998,999,"1,45","-0,0","0,0","1,45","8,2","6,75","5,73",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"19999,39","-5905,0","6970,79","21065,18","21102,9","37,72","109,13",0
1,2,"21102,9","-6155,0","6060,28","21008,18","20966,69","-41,49","59,33",0
2,3,"20966,69","-5975,0","2934,69","17926,38","17990,19","63,81","85,1",0
3,4,"17990,19","-6305,0","6950,94","18636,13","18679,74","43,61","127,56",0
4,5,"18679,74","-5940,0","7943,2","20682,94","20731,84","48,9","178,79",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,73","-0,0","0,0","5,73","3,13","-2,6","-19,24",1
996,997,"3,13","-0,0","0,0","3,13","3,87","0,74","-3,1",1
997,998,"3,87","-0,0","0,0","3,87","5,88","2,01","5,53",1
998,999,"5,88","-0,0","0,0","5,88","2,22","-3,66","-1,13",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"14556,63","-6100,0","7672,4","16129,03","16156,68","27,65","-66,83",0
1,2,"16156,68","-5970,0","3517,74","13704,42","13714,85","10,43","-97,98",0
2,3,"13714,85","-5975,0","8581,27","16321,12","16261,92","-59,2","-126,01",0
3,4,"16261,92","-6155,0","6603,2","16710,12","16773,23","63,11","-59,66",0
4,5,"16773,23","-6065,0","5252,68","15960,91","15952,4","-8,51","20,37",0
...,...,...,...,...,...,...,...,...,...
995,996,"0,48","-0,0","0,0","0,48","5,26","4,78","-4,23",1
996,997,"5,26","-0,0","0,0","5,26","14,41","9,15","9,68",1
997,998,"14,41","-0,0","0,0","14,41","16,94","2,53","14,11",1
998,999,"16,94","-0,0","0,0","16,94","5,52","-11,42","-3,93",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22883,35","-6050,0","5453,86","22287,21","22230,52","-56,69","249,29",0
1,2,"22230,52","-6275,0","3883,29","19838,81","19981,1","142,29","345,12",0
2,3,"19981,1","-6150,0","7525,49","21356,59","21346,06","-10,53","273,17",0
3,4,"21346,06","-6130,0","6948,29","22164,35","22227,3","62,95","280,51",0
4,5,"22227,3","-5815,0","5639,5","22051,8","22112,87","61,07","286,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,47","-0,0","0,0","1,47","15,25","13,78","10,81",1
996,997,"15,25","-0,0","0,0","15,25","6,77","-8,48","-2,84",1
997,998,"6,77","-0,0","0,0","6,77","1,04","-5,73","-11,61",1
998,999,"1,04","-0,0","0,0","1,04","22,11","21,07","14,93",1


### Conjunto de datos de prueba

In [4]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21950,34",-6035,"7006,85","22922,19","22962,99","40,8","172,2",0
1,2,"22962,99",-6105,"6515,76","23373,75","23300,26","-73,49","125,79",0
2,3,"23300,26",-5770,"6438,69","23968,95","24020,7","51,75","97,38",0
3,4,"24020,7",-5915,"5427,88","23533,58","23536,74","3,16","79,51",0
4,5,"23536,74",-6415,"4351,1","21472,84","21473,49","0,65","87,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"15927,66",-6095,"6450,17","16282,83","16239,24","-43,59","114,14",1
996,997,"16239,24",-5855,"6019,11","16403,35","16385,84","-17,51","38,4",1
997,998,"16385,84",-6120,"3750,83","14016,67","13979,49","-37,18","-21,93",1
998,999,"13979,49",-5845,"6725,82","14860,31","14857,03","-3,28","10,77",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22430,06","-6075,0","5976,32","22331,38","22376,0","44,62","-116,83",0
1,2,"22376,0","-6065,0","3965,07","20276,07","20290,81","14,74","-45,27",0
2,3,"20290,81","-5775,0","6982,55","21498,36","21513,87","15,51","-60,91",0
3,4,"21513,87","-6035,0","6956,14","22435,01","22426,74","-8,27","5,59",0
4,5,"22426,74","-6030,0","6377,94","22774,68","22817,15","42,47","36,97",0
...,...,...,...,...,...,...,...,...,...
995,996,"2107,2","-5828,98","6463,64","2741,86","2620,44","-121,42","-167,54",1
996,997,"2620,44","-5955,0","6142,6","2808,04","2771,02","-37,02","-143,07",1
997,998,"2771,02","-5896,0","3108,82","-16,16","25,62","41,78","-150,11",1
998,999,"25,62","-5592,26","6421,37","854,73","896,61","41,88","-83,97",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23654,87","-6005,0","4298,93","21948,8","21972,61","23,81","163,31",0
1,2,"21972,61","-6095,0","4622,96","20500,57","20617,61","117,04","322,76",0
2,3,"20617,61","-6175,0","9067,79","23510,4","23442,51","-67,89","221,84",0
3,4,"23442,51","-5990,0","3410,32","20862,83","20890,7","27,87","233,22",0
4,5,"20890,7","-5915,0","6810,38","21786,08","21803,84","17,76","215,53",0
...,...,...,...,...,...,...,...,...,...
995,996,"10,65","-0,0","0,0","10,65","30,77","20,12","26,54",1
996,997,"30,77","-0,0","0,0","30,77","0,47","-30,3","-3,4",1
997,998,"0,47","-0,0","0,0","0,47","4,08","3,61","0,87",1
998,999,"4,08","-0,0","0,0","4,08","1,82","-2,26","-3,71",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21287,51","-6070,0","7042,91","22260,42","22216,71","-43,71","188,2",0
1,2,"22216,71","-6065,0","4682,57","20834,28","20856,74","22,46","259,79",0
2,3,"20856,74","-6035,0","7234,09","22055,83","22134,3","78,47","272,27",0
3,4,"22134,3","-6120,0","5847,84","21862,14","22001,79","139,65","380,74",0
4,5,"22001,79","-5605,0","3757,23","20154,02","20186,77","32,75","279,92",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,31","-0,0","0,0","3,31","4,22","0,91","-5,75",1
996,997,"4,22","-0,0","0,0","4,22","14,3","10,08","7,99",1
997,998,"14,3","-0,0","0,0","14,3","12,66","-1,64","10,33",1
998,999,"12,66","-0,0","0,0","12,66","2,96","-9,7","-1,54",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18449,36","-6030,0","6911,24","19330,6","19460,91","130,31","181,93",0
1,2,"19460,91","-6125,0","8022,82","21358,73","21278,01","-80,72","73,05",0
2,3,"21278,01","-6065,0","6947,29","22160,3","22222,18","61,88","147,32",0
3,4,"22222,18","-5980,0","7979,31","24221,49","24182,24","-39,25","-32,62",0
4,5,"24182,24","-6245,0","2862,55","20799,79","20848,11","48,32","85,87",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,66","-0,0","0,0","5,66","7,3","1,64","-9,82",1
996,997,"7,3","-0,0","0,0","7,3","10,34","3,04","-0,58",1
997,998,"10,34","-0,0","0,0","10,34","12,28","1,94","-5,24",1
998,999,"12,28","-0,0","0,0","12,28","8,88","-3,4","2,08",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17410,82","-5735,0","7973,05","19648,87","19662,73","13,86","280,74",0
1,2,"19662,73","-6395,0","6009,46","19277,19","19276,39","-0,8","204,46",0
2,3,"19276,39","-5665,0","8990,36","22601,75","22599,67","-2,08","37,4",0
3,4,"22599,67","-6365,0","6085,67","22320,34","22238,26","-82,08","0,27",0
4,5,"22238,26","-6175,0","6515,23","22578,49","22661,17","82,68","148,25",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,64","-0,0","0,0","20,64","3,15","-17,49","-13,37",1
996,997,"3,15","-0,0","0,0","3,15","11,7","8,55","0,13",1
997,998,"11,7","-0,0","0,0","11,7","8,79","-2,91","4,5",1
998,999,"8,79","-0,0","0,0","8,79","13,8","5,01","12,43",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17649,13","-6265,0","12861,14","24245,27","24390,14","144,87","59,32",0
1,2,"24390,14","-6490,0","4415,47","22315,61","22333,73","18,12","59,55",0
2,3,"22333,73","-6055,0","7892,33","24171,06","24129,01","-42,05","78,36",0
3,4,"24129,01","-6280,0","2744,76","20593,77","20587,03","-6,74","119,29",0
4,5,"20587,03","-6115,0","7715,2","22187,23","22232,0","44,77","161,75",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,23","-0,0","0,0","20,23","0,26","-19,97","-14,2",1
996,997,"0,26","-0,0","0,0","0,26","24,53","24,27","2,86",1
997,998,"24,53","-0,0","0,0","24,53","1,94","-22,59","1,24",1
998,999,"1,94","-0,0","0,0","1,94","0,57","-1,37","-1,17",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15272,5","-6010,0","10030,29","19292,79","19280,8","-11,99","292,48",0
1,2,"19280,8","-6205,0","3984,78","17060,58","17094,5","33,92","305,91",0
2,3,"17094,5","-5995,0","7951,55","19051,05","19091,82","40,77","164,52",0
3,4,"19091,82","-6255,0","3983,16","16819,98","16872,08","52,1","215,19",0
4,5,"16872,08","-5965,0","8512,19","19419,27","19433,93","14,66","140,65",0
...,...,...,...,...,...,...,...,...,...
995,996,"16,81","-0,0","0,0","16,81","4,18","-12,63","-0,89",1
996,997,"4,18","-0,0","0,0","4,18","10,01","5,83","5,26",1
997,998,"10,01","-0,0","0,0","10,01","6,14","-3,87","-0,29",1
998,999,"6,14","-0,0","0,0","6,14","14,1","7,96","2,67",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"23299,15","-5900,0","4046,57","21445,72","21419,26","-26,46","496,31",0
1,2,"21419,26","-6310,0","8439,67","23548,93","23527,4","-21,53","356,96",0
2,3,"23527,4","-6055,0","7093,15","24565,55","24485,13","-80,42","233,55",0
3,4,"24485,13","-5925,0","2956,05","21516,18","21525,33","9,15","216,75",0
4,5,"21525,33","-6095,0","6018,92","21449,25","21425,77","-23,48","121,03",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,16","-0,0","0,0","3,16","2,82","-0,34","-5,42",1
996,997,"2,82","-0,0","0,0","2,82","2,05","-0,77","1,14",1
997,998,"2,05","-0,0","0,0","2,05","1,39","-0,66","-3,8",1
998,999,"1,39","-0,0","0,0","1,39","10,38","8,99","-2,67",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22379,58","-5785,0","5690,39","22284,97","22319,5","34,53","144,59",0
1,2,"22319,5","-5980,0","6499,32","22838,82","22799,39","-39,43","171,58",0
2,3,"22799,39","-6000,0","6305,86","23105,25","23145,46","40,21","213,48",0
3,4,"23145,46","-6095,0","5814,71","22865,17","22845,93","-19,24","195,21",0
4,5,"22845,93","-6145,0","6270,76","22971,69","22978,1","6,41","290,05",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,71","-0,0","0,0","2,71","2,61","-0,1","-0,48",1
996,997,"2,61","-0,0","0,0","2,61","5,93","3,32","1,85",1
997,998,"5,93","-0,0","0,0","5,93","13,8","7,87","5,27",1
998,999,"13,8","-0,0","0,0","13,8","6,1","-7,7","-0,73",1


## Limpieza de datos

In [5]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [6]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20247.25,-5580,6051.35,20718.60,20679.11,-39.49,-2.21,0
1,2,20679.11,-6195,7936.54,22420.65,22497.60,76.95,164.76,0
2,3,22497.60,-6165,5995.68,22328.28,22284.53,-43.75,73.79,0
3,4,22284.53,-5720,5031.76,21596.29,21654.39,58.10,181.92,0
4,5,21654.39,-6075,9124.56,24703.95,24903.47,199.52,323.61,0
...,...,...,...,...,...,...,...,...,...
995,996,14567.32,-5990,6995.54,15572.86,15611.86,39.00,-8.11,1
996,997,15611.86,-6405,6226.80,15433.66,15483.99,50.33,14.33,1
997,998,15483.99,-5670,5799.07,15613.06,15537.81,-75.25,6.19,1
998,999,15537.81,-6130,5099.01,14506.82,14515.34,8.52,84.27,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17135.63,"-5975,0",7990.02,19150.65,19183.49,32.84,106.49,0
1,2,19183.49,"-6310,0",7021.06,19894.55,19852.62,-41.93,88.07,0
2,3,19852.62,"-5855,0",7068.29,21065.91,21015.88,-50.03,36.94,0
3,4,21015.88,"-5995,0",5991.07,21011.95,21025.10,13.15,-10.55,0
4,5,21025.10,"-6045,0",6914.83,21894.93,21927.46,32.53,-67.99,0
...,...,...,...,...,...,...,...,...,...
995,996,2535.76,"-6170,0",6732.66,3098.42,3084.18,-14.24,-244.63,1
996,997,3084.18,"-6050,0",5225.99,2260.17,2291.41,31.24,-67.43,1
997,998,2291.41,"-5735,0",7144.70,3701.11,3650.86,-50.25,-129.83,1
998,999,3650.86,"-5915,0",3212.68,948.54,914.60,-33.94,-248.70,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.22,"-5810,0",7001.20,23334.42,23347.02,12.60,-90.27,0
1,2,23347.02,"-6240,0",7270.93,24377.95,24283.54,-94.41,-105.27,0
2,3,24283.54,"-6430,0",3650.03,21503.57,21510.13,6.56,-17.70,0
3,4,21510.13,"-5850,0",6940.97,22601.10,22644.04,42.94,27.91,0
4,5,22644.04,"-6025,0",5910.84,22529.88,22565.92,36.04,80.35,0
...,...,...,...,...,...,...,...,...,...
995,996,20.99,"-0,0",0.00,20.99,7.38,-13.61,5.70,1
996,997,7.38,"-0,0",0.00,7.38,2.99,-4.39,-5.43,1
997,998,2.99,"-0,0",0.00,2.99,14.37,11.38,9.91,1
998,999,14.37,"-0,0",0.00,14.37,23.46,9.09,10.89,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,13689.29,"-6200,0",9859.61,17348.90,17315.01,-33.89,93.19,0
1,2,17315.01,"-5965,0",6578.38,17928.39,18121.96,193.57,276.59,0
2,3,18121.96,"-6170,0",5339.44,17291.40,17357.00,65.60,264.32,0
3,4,17357.00,"-5975,0",6966.25,18348.25,18365.14,16.89,200.58,0
4,5,18365.14,"-6135,0",7964.66,20194.80,20222.38,27.58,192.82,0
...,...,...,...,...,...,...,...,...,...
995,996,14.60,"-0,0",0.00,14.60,3.95,-10.65,-1.28,1
996,997,3.95,"-0,0",0.00,3.95,8.56,4.61,-6.84,1
997,998,8.56,"-0,0",0.00,8.56,1.51,-7.05,-2.43,1
998,999,1.51,"-0,0",0.00,1.51,15.58,14.07,12.31,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21372.17,"-5985,0",7788.90,23176.07,23214.64,38.57,19.95,0
1,2,23214.64,"-5765,0",2969.27,20418.91,20415.11,-3.80,61.14,0
2,3,20415.11,"-6325,0",7390.24,21480.35,21602.29,121.94,144.09,0
3,4,21602.29,"-6190,0",7315.88,22728.17,22691.65,-36.52,217.46,0
4,5,22691.65,"-5690,0",6031.24,23032.89,22983.03,-49.86,170.15,0
...,...,...,...,...,...,...,...,...,...
995,996,17.19,"-0,0",0.00,17.19,26.77,9.58,23.15,1
996,997,26.77,"-0,0",0.00,26.77,11.55,-15.22,10.12,1
997,998,11.55,"-0,0",0.00,11.55,1.45,-10.10,-1.74,1
998,999,1.45,"-0,0",0.00,1.45,8.20,6.75,5.73,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,19999.39,"-5905,0",6970.79,21065.18,21102.90,37.72,109.13,0
1,2,21102.90,"-6155,0",6060.28,21008.18,20966.69,-41.49,59.33,0
2,3,20966.69,"-5975,0",2934.69,17926.38,17990.19,63.81,85.10,0
3,4,17990.19,"-6305,0",6950.94,18636.13,18679.74,43.61,127.56,0
4,5,18679.74,"-5940,0",7943.20,20682.94,20731.84,48.90,178.79,0
...,...,...,...,...,...,...,...,...,...
995,996,5.73,"-0,0",0.00,5.73,3.13,-2.60,-19.24,1
996,997,3.13,"-0,0",0.00,3.13,3.87,0.74,-3.10,1
997,998,3.87,"-0,0",0.00,3.87,5.88,2.01,5.53,1
998,999,5.88,"-0,0",0.00,5.88,2.22,-3.66,-1.13,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,14556.63,"-6100,0",7672.40,16129.03,16156.68,27.65,-66.83,0
1,2,16156.68,"-5970,0",3517.74,13704.42,13714.85,10.43,-97.98,0
2,3,13714.85,"-5975,0",8581.27,16321.12,16261.92,-59.20,-126.01,0
3,4,16261.92,"-6155,0",6603.20,16710.12,16773.23,63.11,-59.66,0
4,5,16773.23,"-6065,0",5252.68,15960.91,15952.40,-8.51,20.37,0
...,...,...,...,...,...,...,...,...,...
995,996,0.48,"-0,0",0.00,0.48,5.26,4.78,-4.23,1
996,997,5.26,"-0,0",0.00,5.26,14.41,9.15,9.68,1
997,998,14.41,"-0,0",0.00,14.41,16.94,2.53,14.11,1
998,999,16.94,"-0,0",0.00,16.94,5.52,-11.42,-3.93,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22883.35,"-6050,0",5453.86,22287.21,22230.52,-56.69,249.29,0
1,2,22230.52,"-6275,0",3883.29,19838.81,19981.10,142.29,345.12,0
2,3,19981.10,"-6150,0",7525.49,21356.59,21346.06,-10.53,273.17,0
3,4,21346.06,"-6130,0",6948.29,22164.35,22227.30,62.95,280.51,0
4,5,22227.30,"-5815,0",5639.50,22051.80,22112.87,61.07,286.84,0
...,...,...,...,...,...,...,...,...,...
995,996,1.47,"-0,0",0.00,1.47,15.25,13.78,10.81,1
996,997,15.25,"-0,0",0.00,15.25,6.77,-8.48,-2.84,1
997,998,6.77,"-0,0",0.00,6.77,1.04,-5.73,-11.61,1
998,999,1.04,"-0,0",0.00,1.04,22.11,21.07,14.93,1


### Conjunto de datos de prueba

In [7]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21950.34,-6035,7006.85,22922.19,22962.99,40.80,172.20,0
1,2,22962.99,-6105,6515.76,23373.75,23300.26,-73.49,125.79,0
2,3,23300.26,-5770,6438.69,23968.95,24020.70,51.75,97.38,0
3,4,24020.70,-5915,5427.88,23533.58,23536.74,3.16,79.51,0
4,5,23536.74,-6415,4351.10,21472.84,21473.49,0.65,87.66,0
...,...,...,...,...,...,...,...,...,...
995,996,15927.66,-6095,6450.17,16282.83,16239.24,-43.59,114.14,1
996,997,16239.24,-5855,6019.11,16403.35,16385.84,-17.51,38.40,1
997,998,16385.84,-6120,3750.83,14016.67,13979.49,-37.18,-21.93,1
998,999,13979.49,-5845,6725.82,14860.31,14857.03,-3.28,10.77,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22430.06,"-6075,0",5976.32,22331.38,22376.00,44.62,-116.83,0
1,2,22376.00,"-6065,0",3965.07,20276.07,20290.81,14.74,-45.27,0
2,3,20290.81,"-5775,0",6982.55,21498.36,21513.87,15.51,-60.91,0
3,4,21513.87,"-6035,0",6956.14,22435.01,22426.74,-8.27,5.59,0
4,5,22426.74,"-6030,0",6377.94,22774.68,22817.15,42.47,36.97,0
...,...,...,...,...,...,...,...,...,...
995,996,2107.20,"-5828,98",6463.64,2741.86,2620.44,-121.42,-167.54,1
996,997,2620.44,"-5955,0",6142.60,2808.04,2771.02,-37.02,-143.07,1
997,998,2771.02,"-5896,0",3108.82,-16.16,25.62,41.78,-150.11,1
998,999,25.62,"-5592,26",6421.37,854.73,896.61,41.88,-83.97,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23654.87,"-6005,0",4298.93,21948.80,21972.61,23.81,163.31,0
1,2,21972.61,"-6095,0",4622.96,20500.57,20617.61,117.04,322.76,0
2,3,20617.61,"-6175,0",9067.79,23510.40,23442.51,-67.89,221.84,0
3,4,23442.51,"-5990,0",3410.32,20862.83,20890.70,27.87,233.22,0
4,5,20890.70,"-5915,0",6810.38,21786.08,21803.84,17.76,215.53,0
...,...,...,...,...,...,...,...,...,...
995,996,10.65,"-0,0",0.00,10.65,30.77,20.12,26.54,1
996,997,30.77,"-0,0",0.00,30.77,0.47,-30.30,-3.40,1
997,998,0.47,"-0,0",0.00,0.47,4.08,3.61,0.87,1
998,999,4.08,"-0,0",0.00,4.08,1.82,-2.26,-3.71,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21287.51,"-6070,0",7042.91,22260.42,22216.71,-43.71,188.20,0
1,2,22216.71,"-6065,0",4682.57,20834.28,20856.74,22.46,259.79,0
2,3,20856.74,"-6035,0",7234.09,22055.83,22134.30,78.47,272.27,0
3,4,22134.30,"-6120,0",5847.84,21862.14,22001.79,139.65,380.74,0
4,5,22001.79,"-5605,0",3757.23,20154.02,20186.77,32.75,279.92,0
...,...,...,...,...,...,...,...,...,...
995,996,3.31,"-0,0",0.00,3.31,4.22,0.91,-5.75,1
996,997,4.22,"-0,0",0.00,4.22,14.30,10.08,7.99,1
997,998,14.30,"-0,0",0.00,14.30,12.66,-1.64,10.33,1
998,999,12.66,"-0,0",0.00,12.66,2.96,-9.70,-1.54,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18449.36,"-6030,0",6911.24,19330.60,19460.91,130.31,181.93,0
1,2,19460.91,"-6125,0",8022.82,21358.73,21278.01,-80.72,73.05,0
2,3,21278.01,"-6065,0",6947.29,22160.30,22222.18,61.88,147.32,0
3,4,22222.18,"-5980,0",7979.31,24221.49,24182.24,-39.25,-32.62,0
4,5,24182.24,"-6245,0",2862.55,20799.79,20848.11,48.32,85.87,0
...,...,...,...,...,...,...,...,...,...
995,996,5.66,"-0,0",0.00,5.66,7.30,1.64,-9.82,1
996,997,7.30,"-0,0",0.00,7.30,10.34,3.04,-0.58,1
997,998,10.34,"-0,0",0.00,10.34,12.28,1.94,-5.24,1
998,999,12.28,"-0,0",0.00,12.28,8.88,-3.40,2.08,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17410.82,"-5735,0",7973.05,19648.87,19662.73,13.86,280.74,0
1,2,19662.73,"-6395,0",6009.46,19277.19,19276.39,-0.80,204.46,0
2,3,19276.39,"-5665,0",8990.36,22601.75,22599.67,-2.08,37.40,0
3,4,22599.67,"-6365,0",6085.67,22320.34,22238.26,-82.08,0.27,0
4,5,22238.26,"-6175,0",6515.23,22578.49,22661.17,82.68,148.25,0
...,...,...,...,...,...,...,...,...,...
995,996,20.64,"-0,0",0.00,20.64,3.15,-17.49,-13.37,1
996,997,3.15,"-0,0",0.00,3.15,11.70,8.55,0.13,1
997,998,11.70,"-0,0",0.00,11.70,8.79,-2.91,4.50,1
998,999,8.79,"-0,0",0.00,8.79,13.80,5.01,12.43,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17649.13,"-6265,0",12861.14,24245.27,24390.14,144.87,59.32,0
1,2,24390.14,"-6490,0",4415.47,22315.61,22333.73,18.12,59.55,0
2,3,22333.73,"-6055,0",7892.33,24171.06,24129.01,-42.05,78.36,0
3,4,24129.01,"-6280,0",2744.76,20593.77,20587.03,-6.74,119.29,0
4,5,20587.03,"-6115,0",7715.20,22187.23,22232.00,44.77,161.75,0
...,...,...,...,...,...,...,...,...,...
995,996,20.23,"-0,0",0.00,20.23,0.26,-19.97,-14.20,1
996,997,0.26,"-0,0",0.00,0.26,24.53,24.27,2.86,1
997,998,24.53,"-0,0",0.00,24.53,1.94,-22.59,1.24,1
998,999,1.94,"-0,0",0.00,1.94,0.57,-1.37,-1.17,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15272.50,"-6010,0",10030.29,19292.79,19280.80,-11.99,292.48,0
1,2,19280.80,"-6205,0",3984.78,17060.58,17094.50,33.92,305.91,0
2,3,17094.50,"-5995,0",7951.55,19051.05,19091.82,40.77,164.52,0
3,4,19091.82,"-6255,0",3983.16,16819.98,16872.08,52.10,215.19,0
4,5,16872.08,"-5965,0",8512.19,19419.27,19433.93,14.66,140.65,0
...,...,...,...,...,...,...,...,...,...
995,996,16.81,"-0,0",0.00,16.81,4.18,-12.63,-0.89,1
996,997,4.18,"-0,0",0.00,4.18,10.01,5.83,5.26,1
997,998,10.01,"-0,0",0.00,10.01,6.14,-3.87,-0.29,1
998,999,6.14,"-0,0",0.00,6.14,14.10,7.96,2.67,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,23299.15,"-5900,0",4046.57,21445.72,21419.26,-26.46,496.31,0
1,2,21419.26,"-6310,0",8439.67,23548.93,23527.40,-21.53,356.96,0
2,3,23527.40,"-6055,0",7093.15,24565.55,24485.13,-80.42,233.55,0
3,4,24485.13,"-5925,0",2956.05,21516.18,21525.33,9.15,216.75,0
4,5,21525.33,"-6095,0",6018.92,21449.25,21425.77,-23.48,121.03,0
...,...,...,...,...,...,...,...,...,...
995,996,3.16,"-0,0",0.00,3.16,2.82,-0.34,-5.42,1
996,997,2.82,"-0,0",0.00,2.82,2.05,-0.77,1.14,1
997,998,2.05,"-0,0",0.00,2.05,1.39,-0.66,-3.80,1
998,999,1.39,"-0,0",0.00,1.39,10.38,8.99,-2.67,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22379.58,"-5785,0",5690.39,22284.97,22319.50,34.53,144.59,0
1,2,22319.50,"-5980,0",6499.32,22838.82,22799.39,-39.43,171.58,0
2,3,22799.39,"-6000,0",6305.86,23105.25,23145.46,40.21,213.48,0
3,4,23145.46,"-6095,0",5814.71,22865.17,22845.93,-19.24,195.21,0
4,5,22845.93,"-6145,0",6270.76,22971.69,22978.10,6.41,290.05,0
...,...,...,...,...,...,...,...,...,...
995,996,2.71,"-0,0",0.00,2.71,2.61,-0.10,-0.48,1
996,997,2.61,"-0,0",0.00,2.61,5.93,3.32,1.85,1
997,998,5.93,"-0,0",0.00,5.93,13.80,7.87,5.27,1
998,999,13.80,"-0,0",0.00,13.80,6.10,-7.70,-0.73,1


<!-- TODO -->

## Selección de características

In [8]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [9]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,-39.49,0
1,76.95,0
2,-43.75,0
3,58.10,0
4,199.52,0
...,...,...
995,39.00,1
996,50.33,1
997,-75.25,1
998,8.52,1


,Variacion,Fugando combustible
0,32.84,0
1,-41.93,0
2,-50.03,0
3,13.15,0
4,32.53,0
...,...,...
995,-14.24,1
996,31.24,1
997,-50.25,1
998,-33.94,1


,Variacion,Fugando combustible
0,12.60,0
1,-94.41,0
2,6.56,0
3,42.94,0
4,36.04,0
...,...,...
995,-13.61,1
996,-4.39,1
997,11.38,1
998,9.09,1


,Variacion,Fugando combustible
0,-33.89,0
1,193.57,0
2,65.60,0
3,16.89,0
4,27.58,0
...,...,...
995,-10.65,1
996,4.61,1
997,-7.05,1
998,14.07,1


,Variacion,Fugando combustible
0,38.57,0
1,-3.80,0
2,121.94,0
3,-36.52,0
4,-49.86,0
...,...,...
995,9.58,1
996,-15.22,1
997,-10.10,1
998,6.75,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,37.72,0
1,-41.49,0
2,63.81,0
3,43.61,0
4,48.90,0
...,...,...
995,-2.60,1
996,0.74,1
997,2.01,1
998,-3.66,1


,Variacion,Fugando combustible
0,27.65,0
1,10.43,0
2,-59.20,0
3,63.11,0
4,-8.51,0
...,...,...
995,4.78,1
996,9.15,1
997,2.53,1
998,-11.42,1


,Variacion,Fugando combustible
0,-56.69,0
1,142.29,0
2,-10.53,0
3,62.95,0
4,61.07,0
...,...,...
995,13.78,1
996,-8.48,1
997,-5.73,1
998,21.07,1


### Conjunto de datos de prueba

In [10]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,40.80,0
1,-73.49,0
2,51.75,0
3,3.16,0
4,0.65,0
...,...,...
995,-43.59,1
996,-17.51,1
997,-37.18,1
998,-3.28,1


,Variacion,Fugando combustible
0,44.62,0
1,14.74,0
2,15.51,0
3,-8.27,0
4,42.47,0
...,...,...
995,-121.42,1
996,-37.02,1
997,41.78,1
998,41.88,1


,Variacion,Fugando combustible
0,23.81,0
1,117.04,0
2,-67.89,0
3,27.87,0
4,17.76,0
...,...,...
995,20.12,1
996,-30.30,1
997,3.61,1
998,-2.26,1


,Variacion,Fugando combustible
0,-43.71,0
1,22.46,0
2,78.47,0
3,139.65,0
4,32.75,0
...,...,...
995,0.91,1
996,10.08,1
997,-1.64,1
998,-9.70,1


,Variacion,Fugando combustible
0,130.31,0
1,-80.72,0
2,61.88,0
3,-39.25,0
4,48.32,0
...,...,...
995,1.64,1
996,3.04,1
997,1.94,1
998,-3.40,1


...

,Variacion,Fugando combustible
0,13.86,0
1,-0.80,0
2,-2.08,0
3,-82.08,0
4,82.68,0
...,...,...
995,-17.49,1
996,8.55,1
997,-2.91,1
998,5.01,1


,Variacion,Fugando combustible
0,144.87,0
1,18.12,0
2,-42.05,0
3,-6.74,0
4,44.77,0
...,...,...
995,-19.97,1
996,24.27,1
997,-22.59,1
998,-1.37,1


,Variacion,Fugando combustible
0,-11.99,0
1,33.92,0
2,40.77,0
3,52.10,0
4,14.66,0
...,...,...
995,-12.63,1
996,5.83,1
997,-3.87,1
998,7.96,1


,Variacion,Fugando combustible
0,-26.46,0
1,-21.53,0
2,-80.42,0
3,9.15,0
4,-23.48,0
...,...,...
995,-0.34,1
996,-0.77,1
997,-0.66,1
998,8.99,1


,Variacion,Fugando combustible
0,34.53,0
1,-39.43,0
2,40.21,0
3,-19.24,0
4,6.41,0
...,...,...
995,-0.10,1
996,3.32,1
997,7.87,1
998,-7.70,1


<!-- TODO -->

## Unificación de conjunto de datos

In [11]:
complete_train_dataframe = pd.concat([data_dict[key]['train_dataframe'] for key in data_dict])
# complete_train_dataframe = data_dict['case_0001']['train_dataframe']

X_train_complete = complete_train_dataframe.drop(columns='Fugando combustible')
y_train_complete = complete_train_dataframe['Fugando combustible']

## Implementación de modelos

### Definición de métricas

In [12]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = 0
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

### Aplicación de `GridSearch` a modelos

In [34]:
from sklearn.model_selection import GridSearchCV

def eval_classic_model(base_classifier, param_grid, cv = 5) -> None:
    grid_search = GridSearchCV(
        estimator = base_classifier,
        param_grid = param_grid,
        cv = cv
    )

    grid_search.fit(X_train_complete, y_train_complete)

    print(f"Best parameters found: {grid_search.best_params_}")

    result_dict = {}

    best_classifier = grid_search.best_estimator_
    for case in data_dict:
        leak_value = data_dict[case]['leak_value']
        test_dataframe = data_dict[case]['test_dataframe']

        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']

        best_classifier = grid_search.best_estimator_
        y_pred = best_classifier.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        leak_delay = leak_detection_delay(y_test, y_pred)

        result_dict[leak_value] = [accuracy, leak_delay]
    
    display(pd.DataFrame.from_dict(
        data = result_dict,
        orient = 'index',
        columns = ['accuracy_score', 'leak_detection_delay']
    ))

### Modelos clásicos

#### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

eval_classic_model(rf_classifier, rf_param_grid)

#### Naive Bayes Classifier

In [91]:
from sklearn.naive_bayes import GaussianNB

gnb_classifier = GaussianNB()

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

eval_classic_model(gnb_classifier, gnb_param_grid)

Best parameters found: {'var_smoothing': 1e-05}


,accuracy_score,leak_detection_delay
0.013000,0.500,-495
0.054929,0.596,-493
0.096857,0.613,-494
0.138786,0.582,-492
0.180714,0.573,-483
0.222643,0.546,-493
0.264571,0.544,-483
0.306500,0.519,-492
0.348429,0.502,-494
0.390357,0.494,-495


#### KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

eval_classic_model(knn_classifier, knn_param_grid)

#### XGBoost Classifier

In [ ]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier()

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

eval_classic_model(xgb_classifier, xgb_param_grid)

### Múltiples modelos de _Deep Learning_

In [12]:
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 4,
        verbose = 0
    )

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    test_accuracy = accuracy_score(y_test, y_pred)

    test_leak_delay = leak_detection_delay(y_test, y_pred)

    print(f'({case}):\tAccuracy {test_accuracy:.4f}\tLeak delay: {test_leak_delay}')

(case_0000):	Accuracy 0.5640	Leak delay: -495
(case_0001):	Accuracy 0.6250	Leak delay: -491
(case_0002):	Accuracy 0.6830	Leak delay: -495
(case_0003):	Accuracy 0.7230	Leak delay: -494
(case_0004):	Accuracy 0.7770	Leak delay: -494
(case_0005):	Accuracy 0.8190	Leak delay: -494
(case_0006):	Accuracy 0.8010	Leak delay: -494
(case_0007):	Accuracy 0.8030	Leak delay: -494
(case_0008):	Accuracy 0.8290	Leak delay: -494
(case_0009):	Accuracy 0.8120	Leak delay: -493
(case_0010):	Accuracy 0.8080	Leak delay: -492
(case_0011):	Accuracy 0.8480	Leak delay: -493
(case_0012):	Accuracy 0.7630	Leak delay: -490
(case_0013):	Accuracy 0.7920	Leak delay: -494
(case_0014):	Accuracy 0.7810	Leak delay: -491


### Único modelo

In [13]:
import time

In [14]:
X_train_complete_np = X_train_complete.values.reshape(-1, X_train_complete.shape[1], 1)
y_train_complete_np = y_train_complete.values

In [15]:
avg_time = 0
for i in range(20):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train_complete.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    start = time.time()
    history = model.fit(
        x = X_train_complete_np,
        y = y_train_complete_np,
        epochs = 50,
        batch_size = 1024,
        verbose = 0
    )
    avg_time += (time.time() - start)

print(avg_time / 20)

2.4470803260803224


In [16]:
# Convertir DataFrame y Series a numpy arrays
X_train_complete_np = X_train_complete.values.reshape(-1, X_train_complete.shape[1], 1)
y_train_complete_np = y_train_complete.values

# Crear el dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_complete_np, y_train_complete_np))
train_dataset = train_dataset.shuffle(buffer_size=2048).batch(2048).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

avg_time = 0
for i in range(20):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train_complete.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    start = time.time()
    history = model.fit(
        train_dataset,
        epochs = 50,
        verbose = 0
    )
    avg_time += (time.time() - start)

print(avg_time / 20)

2.17256520986557


In [17]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.16.1
No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.
